In [2]:
import os
#os.chdir('./Notebooks')
import spmfunctions.misc_tools
from spmfunctions.misc_tools import *
from spmfunctions.read_data import *
from spmfunctions.plotting import *
from spmfunctions.counts import *
from spmfunctions.video_processing import *
import pandas as pd
import importlib
from typing import Dict, Tuple, List, Set

In [ ]:
input_jpg = "C:\\Users\\rhansen\\Documents\\Python\\SPMs\\Intersections\\SH-55 & Banks-Lowman\\Configuration\\vid_bg.jpg"
output_mp4 = "C:\\Users\\rhansen\\Documents\\Python\\SPMs\\Intersections\\SH-55 & Banks-Lowman\\Configuration\\vid_bg.mp4"

create_video_from_image(input_jpg, output_mp4)

Video saved: C:\Users\rhansen\OneDrive - Idaho Transportation Department\Documents\Python\SPMs\Intersections\SH-55 & Banks-Lowman\Configuration\vid_bg.mp4


In [4]:
#pairs = [(26, 33), (17, 46), (24,38), (11,51), (13,52)]
pairs = [(33,35)]
pairs = [(56,33), (57, 46), (58,35)]
pairs = [(16,32)]
#pairs=[(33,35), (36,37), (41,43), (44,45)]
path_out = "C:\\Users\\rhansen\\Documents\\Python\\SPMs\\Intersections\\SH-55 & Banks-Lowman\\Plotting\\Detector_Comparison_2025_12_07-2025_12_15-crx_fail.html"
#path_out = "C:\\Users\\rhansen\\Documents\\Python\\SPMs\\Intersections\\SH-44 & Cemetery\\Plotting\\Detector_Comparison_2025-09-18.html"


create_detector_comparison_plot(df_raw, pairs, path_out)


In [21]:
importlib.reload(spmfunctions.misc_tools)
from spmfunctions.misc_tools import *

In [3]:
spm_dir = 'C:/users/rhansen/documents/python/spms/'
intersection = 'SH-55 & Banks-Lowman'
#intersection = 'Franklin & Smeed'
#intersection = "SH-44 & Cemetery"
cfg_dir = os.path.join(spm_dir, "intersections", intersection, "configuration")
pickle_dir = os.path.join(spm_dir, "intersections", intersection, "data", "dataframes")
plt_dir = os.path.join(spm_dir, "intersections", intersection, "plotting")
#int_cfg= read_int_cfg(cfg_dir)
date_range = '2025_12_07-2025_12_14'

df_raw= read_and_process_pickles(pickle_dir, date_range)
df_comb = comb_gyr_det(df_raw)

In [12]:
sensor_dict = {
    'Sensor1' : (4,(33,38,39,40,42,43,45)),
    'Sensor2' : (5,(46,47,48,49,53,55,60,61,64)),
    'Sensor3' : (3,(34,35,36))
}

sim_res = find_simultaneous_events_by_sensor(df_raw, sensor_dict)

In [32]:
df_s_pvt.reset_index().TS_start.dt.floor('H').unique().shape[0]/24

C:\Users\rhansen\AppData\Local\Temp\ipykernel_12252\1653815427.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_s_pvt.reset_index().TS_start.dt.floor('H').unique().shape[0]/24


4.625

In [27]:
sensor='Sensor3'

df_s = df_comb.loc[((df_comb.TS_start.isin(sim_res[sensor]))&
                    (df_comb.Code.isin([81,82]))&
                    (df_comb.TS_start.dt.hour.isin([7,8,9,10,11,12,13,14,15,16,17,18,19,20]))&
                    (df_comb.ID.isin(sensor_dict[sensor][1]))), ['TS_start', 'ID','Duration']]

#pd.merge(df_s.groupby('TS_start').min(), df_s.groupby('TS_start').max(), on='TS_start', suffixes=('_min', '_max'))
df_s_pvt = df_s.pivot_table(index='TS_start', columns='ID', values='Duration', aggfunc='min')

In [33]:
df_s_pvt

ID,34,35,36
TS_start,,,
2025-06-08 10:05:03.300,0.5,0.4,0.4
2025-06-08 10:46:45.000,0.4,0.5,0.3
2025-06-10 08:15:14.000,168.2,166.2,164.3
2025-06-10 09:04:55.700,123.1,290.4,127.3
2025-06-10 09:19:59.400,307.0,305.3,162.9
...,...,...,...
2025-08-06 11:14:52.300,1.0,1.0,1.0
2025-08-06 11:15:00.800,24.0,24.1,24.1
2025-08-06 11:15:36.000,1.0,1.0,1.0


In [11]:
VariableSensorDict = Dict[str, Tuple[int, Tuple[int, ...]]]

def find_simultaneous_events_by_sensor(
    df: pd.DataFrame,
    sensor_dict: VariableSensorDict,
) -> Dict[str, List[pd.Timestamp]]:
    """
    Identifies occurrences where Code=82 events sharing the EXACT same TS_start 
    timestamp (0.1s granularity) meet a sensor's specified minimum unique event count.

    Args:
        df: DataFrame with 'TS_start' (datetime), 'Code' (int), 'ID' (int) columns.
        sensor_dict: Dictionary where keys are target sensor names (str) and values 
                     are a tuple (MinCount, (ID1, ID2, ...)).

    Returns:
        A dictionary where keys are the target sensor names (str) and values are
        a list of unique TS_start timestamps (pd.Timestamp) where that sensor's
        pattern qualified.
    """
    # 1. Filter the DataFrame for Code=82 events
    df_82 = df[df['Code'] == 82][['TS_start', 'ID']].copy()
    
    if df_82.empty:
        print("No Code=82 events found in the DataFrame.")
        return {}

    # 2. Iterate through each Target Sensor and find matching TS_start values
    final_result_dict: Dict[str, List[pd.Timestamp]] = {}

    for target_sensor, (min_count, target_ids) in sensor_dict.items():
        
        # Filter the Code=82 data to include only the IDs relevant to the current sensor
        # Note: Even if an ID belongs to multiple sensors, we only care about 
        # events associated with *this* target sensor's ID list.
        df_sensor = df_82[df_82['ID'].isin(target_ids)].copy()
        
        if df_sensor.empty:
            final_result_dict[target_sensor] = []
            continue

        # 3. Group by TS_start and Count Unique IDs
        # We need the number of unique IDs (events) that occurred at that exact TS_start.
        
        # Group by TS_start and calculate the count of unique IDs for that moment.
        # This count represents the number of simultaneous events detected by this sensor's IDs.
        simultaneity_counts = df_sensor.groupby('TS_start')['ID'].nunique()
        
        # 4. Apply the MinCount Threshold
        
        # Filter the timestamps where the count meets or exceeds the minimum requirement
        qualifying_timestamps = simultaneity_counts[
            simultaneity_counts >= min_count
        ].index
        
        # Store the sorted list of qualifying timestamps
        final_result_dict[target_sensor] = sorted(list(qualifying_timestamps))

    return final_result_dict

In [ ]:
import requests

def download_mp4(url: str, save_path: str, username: str = None, password: str = None):
    """
    Download an MP4 file from a given URL and save it locally.

    :param url: The full URL to the MP4 file (e.g. http://192.168.1.100/video.mp4)
    :param save_path: The local path to save the file (e.g. output.mp4)
    """
    auth = (username, password) if username and password else None
    response = requests.get(url, stream=True, auth=auth)
    response.raise_for_status()  # Raise error if download fails

    with open(save_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:  # filter out keep-alive chunks
                f.write(chunk)

    print(f"Downloaded file saved to {save_path}")


if __name__ == "__main__":
    # Example usage:
    url = "http://10.37.23.203:8020/vehicle_event/baysgebmwtm.mp4"  # Replace with your IP/file path
    username = "admin"
    password = "123456"
    save_path = "downloaded_video.mp4"
    download_mp4(url, save_path, username, password)

In [10]:
df_comb[df_comb['Code']==1].groupby(['ID']).Split.describe()

,count,mean,std,min,25%,50%,75%,max
ID,,,,,,,,
2,538.0,98.710409,123.725692,10.0,42.9,71.05,107.1,1265.5
3,201.0,63.217413,244.825584,8.1,17.4,24.40,35.7,3370.5
4,201.0,22.288557,12.832175,9.1,11.4,16.50,30.3,72.4
6,538.0,98.710409,123.725692,10.0,42.9,71.05,107.1,1265.5
7,485.0,30.300206,12.251078,10.5,19.9,28.20,40.5,86.3
8,241.0,15.268050,4.936515,8.6,10.6,14.30,19.0,25.6
11,189.0,3.176720,3.087033,2.4,2.4,2.40,2.4,27.6
16,33.0,7.633333,3.223514,5.1,5.1,5.10,10.6,13.1


In [4]:
df_comb[df_comb['Code']==82].groupby(['ID']).Duration.describe()

,count,mean,std,min,25%,50%,75%,max
ID,,,,,,,,
9,200.0,21.083500,26.867493,0.8,2.475,6.80,29.075,108.9
11,322.0,5.049068,3.763791,0.8,3.000,4.30,6.000,36.1
12,7.0,2.928571,2.840607,1.6,1.700,1.70,2.250,9.3
13,308.0,5.080519,3.309981,0.8,3.400,4.30,6.000,30.7
17,1278.0,6.303599,11.869544,0.8,1.600,2.50,5.000,101.0
22,150.0,15.624667,21.596492,0.8,4.200,6.60,14.100,120.1
24,574.0,21.731010,26.811555,0.8,3.300,9.15,29.350,125.9
26,965.0,7.497927,13.121587,0.8,1.600,2.50,5.800,116.4
31,209.0,4.287560,9.317546,0.8,1.600,2.50,4.100,112.0


In [5]:
df_r1r2 = add_r1r2(df_comb)

In [7]:
df_filt = df_r1r2[['Cycle_start', 'R1_Ph', 'R2_Ph']].drop_duplicates()

In [8]:
df_filt.groupby('R1_Ph').count()

,Cycle_start,R2_Ph
R1_Ph,,
"11,3",1,1
2,179,179
"2,11",13,13
"2,11,11,3",1,1
"2,11,3",64,64
"2,2",1,1
"2,2,11,3",1,1
"2,2,4",1,1
"2,3",15,15


In [3]:
# For df_comb
transforms = [(comb_gyr_det, {})]
df = read_and_process_pickles(pickle_dir, date_range, transforms)

In [10]:
plot_coord(df_comb, pd.Series([]), 'out.html')

ValueError: cannot set a frame with no defined index and a scalar

In [15]:
for date in df.Cycle_start.dt.date.unique():
    formatted_date = date.strftime('%Y_%m_%d')
    nearest_date = get_nearest_date(int_cfg['Ring-Barrier'].index, date)
    both_str = int_cfg['Ring-Barrier'].loc[nearest_date, 'B']
    both = list(map(int, both_str.split(','))) if pd.notna(both_str) else []
    det_srs = int_cfg['Arrivals'].T.dropna(how='any')[nearest_date]
    plot_coord(df[df.Cycle_start.dt.date == date], det_srs, plt_dir, both=both, sfx=formatted_date)
    print(f"Coordination / Split Diagram plot for {formatted_date} saved at {plt_dir}.")

c:\Users\rhansen\OneDrive - Idaho Transportation Department\Documents\Python\SPMs\Notebooks\spmfunctions\plotting.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ring = pd.concat([df_ring] + [df for df in temp_dfs if not df.empty], ignore_index=True)


ValueError: You are trying to merge on int64 and datetime64[ns] columns for key 'Cycle_start'. If you wish to proceed you should use pd.concat

In [4]:
date = df.Cycle_start.dt.date.unique()[0]
formatted_date = date.strftime('%Y_%m_%d')
nearest_date = get_nearest_date(int_cfg['Ring-Barrier'].index, date)
both_str = int_cfg['Ring-Barrier'].loc[nearest_date, 'B']
both = list(map(int, both_str.split(','))) if pd.notna(both_str) else []
det_srs = int_cfg['Arrivals'].T.dropna(how='any')[nearest_date]
#df = add_r1r2(df)

In [ ]:
import warnings


df = read_and_process_pickles(pickle_dir, date_range, transforms)
df = add_r1r2(df)

df_comb = df.copy()

resdict = {}
for r in [1, 2]:
    seq = df_comb.loc[:, f'R{r}_Ph'].dropna().unique()
    df_seq = pd.DataFrame([list(x) for x in seq], index=seq)
    df_seq.columns = [f' ({x})' for x in range(1, len(df_seq.columns) + 1)]
    df_seq = df_seq.astype('str') + df_seq.columns
    cols = []
    for c in df_seq.columns:
        for x in (df_seq.loc[:, c].dropna().unique()):
            for y in ['G', 'Y', 'Rc']:
                if 'None' not in x:
                    cols.append(f'{y} {x}')

    df_ring = pd.DataFrame(columns=cols)

    temp_dfs = []
    for seq, row in df_seq.iterrows():
        idx = []
        for v in row:
            if v.startswith('None'):
                continue
            for st in ['G', 'Y', 'Rc']:
                idx.append(f'{st} {v}')

        df_f = df_comb[(df_comb.ID.isin([int(x) for x in seq])) & (df_comb.Code.isin([1, 8, 10])) &
                        (df_comb.loc[:, f'R{r}_Ph'] == seq)]

        for cs, dfgb in df_f.groupby('Cycle_start'):
            srs = dfgb.Duration
            if len(srs) != len(idx):
                continue
            srs.index = idx
            srs.name = cs
            temp_dfs.append(pd.DataFrame(srs).T)
            #df_ring = pd.concat([df_ring, pd.DataFrame(srs).T])
    
    

    df_ring = pd.concat([df_ring] + [df for df in temp_dfs if not df.empty])

    df_ring = df_ring.fillna(0)
    df_ring.index.name = 'Cycle_start'
    df_ring = pd.merge(df_ring, df_comb.loc[:, ['Cycle_start', 'Coord_plan']], how='left', on='Cycle_start').drop_duplicates()
    df_ring.loc[:, 'Coord_plan'] = df_ring.Coord_plan

    resdict[r] = df_ring


C:\Users\rhansen\AppData\Local\Temp\ipykernel_19736\1233889161.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_ring = pd.concat([df_ring] + [df for df in temp_dfs if not df.empty])
C:\Users\rhansen\AppData\Local\Temp\ipykernel_19736\1233889161.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ring = pd.concat([df_ring] + [df for df in temp_dfs if not df.empty])
C:\Users\rhansen\AppData\Local\Temp\ipykernel_19736\1233889161.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecate

In [26]:
pd.DataFrame(srs).T

,G 1 (1),Y 1 (1),Rc 1 (1),G 1 (2),Y 1 (2),Rc 1 (2),G 2 (3),Y 2 (3),Rc 2 (3),G 4 (4),Y 4 (4),Rc 4 (4)
2025-01-29 15:02:43.500,10.8,4.6,3.1,7.1,4.0,1.6,17.1,5.4,0.5,10.9,4.0,2.1
